# Test Environment

In [280]:
# Modul um Würfel zu simulieren
import random

# Modul zum vereinfachten Auslesen und Bearbeiten von Datentabellen
import pandas as pd
import os
from pathlib import Path

# Um User Input zu verarbeiten
import regex as re

## Aux Funktionen zur Würfelsimulation

In [281]:
def w20(anzahl: int = 1) -> list[int]:
    """
    Simulation eines W20 Würfels. 

    : anzahl : Integer, Anzahl der Würfe
    : return : Liste mit Ergebnissen (Integer 1-20)
    """

    assert anzahl > 0

    wuerfe = [random.randint(1, 20) for _ in range(anzahl)]

    #kritischer_erfolg(wuerfe)

    if len(wuerfe) == 1:
        return wuerfe[0]

    return wuerfe


def w6(anzahl: int = 1) -> list[int]:
    """
    Simulation eines W6 Würfels. 

    :anzahl: Integer, Anzahl der Würfe
    : return : Liste mit Ergebnissen (Integer 1-6)
    """

    assert anzahl > 0

    wuerfe = [random.randint(1, 6) for _ in range(anzahl)]

    #kritischer_erfolg(wuerfe)

    if len(wuerfe) == 1:
        return wuerfe[0]

    return sum(wuerfe)


def kritischer_erfolg(wuerfe: list[int]) -> None:
    """
    Prüft bei Wurfproben ob ein kritischer (Miss-)Erfolg erzielt wurde.

    : wuerfe : Ergebnisliste, enthält Integer
    : return : None
    """

    if len(wuerfe) == 1:
        if 1 in wuerfe:
            print("Kritischer Erfolg!")
        elif 20 in wuerfe:
            print("Kritischer Misserfolg!")
    
    elif len(wuerfe) > 1:
        anzahl_einsen = [x for x in wuerfe if x == 1]
        anzahl_zwanziger = [x for x in wuerfe if x == 20]

        if len(anzahl_einsen) >= 2:
            print("Kritischer Erfolg der Fertigkeitsprobe!")

        elif len(anzahl_zwanziger) >= 2:
            print("Kritischer Misserfolg der Fertigkeitsprobe!")
    
    else:
        return None

## Aux Funktionen zur Datenverwaltung

In [282]:
def datei_zu_dataframe(path: str, index_column: int = 0) -> pd.DataFrame:
    """
    Liest eine excel Datei ein

    : path : String, Pfad zur Datei
    : index_column : Integer
    : return : pandas Dataframe
    """

    dataframe = pd.read_excel(path, index_col=index_column)
    dataframe.dropna(inplace=True)

    if "Eigenschaftsprobe" in dataframe.columns:
        dataframe["Eigenschaftsprobe"] = dataframe.Eigenschaftsprobe.apply(lambda x: x.split("/"))
    elif "Leiteigenschaft" in dataframe.columns:
        dataframe["Leiteigenschaft"] = dataframe.Leiteigenschaft.apply(lambda x: x.split("/"))

    return dataframe


def datei_zu_dict(path: str) -> dict:
    """
    Liest eine excel Tabelle ein und gibt ein dictionary aus, wobei die Spaltenname der Tabelle
    die keys für das dictionary darstellen werden. 
    
    : path : String, Pfad zur Datei
    : return : dict
    """

    frame = datei_zu_dataframe(path)
    framedict = frame.T.to_dict(orient="records")[0]

    return framedict


def zeile_zu_dict(frame: pd.DataFrame, zeile: str, art: str = None):
    """
    Bildet eine einzelne Teile eines Dataframes als dictionary ab

    : frame : pd.Dataframe
    : zeile : String, muss Indexname einer Zeile sein
    : return : dict
    """

    assert zeile in frame.T.columns

    framedict = frame.T[zeile].to_dict()

    if art:
        framedict[art] = zeile

    return framedict


def zeilen_zu_dict(frame: pd.DataFrame):
    """
    
    """

    zeilendict = {}
    zauberliste = [x for x in frame.T.columns]

    for zauber in zauberliste:
        zeilendict[zauber] = zeile_zu_dict(frame, zauber)
    
    return zeilendict

In [283]:
# Einlesen der Datentabelle zu den Fertigkeiten
dsa5_talente = datei_zu_dataframe("data/dsa5_talente.xlsx", index_column=1)
dsa5_kampftechnik = datei_zu_dataframe("data/kampftechnik.xlsx")

dsa5_waffen = datei_zu_dataframe("data/waffen.xlsx")
dsa5_ruestungen = datei_zu_dataframe("data/ruestungen.xlsx")
dsa5_zauber = datei_zu_dataframe("data/zauber.xlsx")

# Glossar einlesen und mit dem gespiegelten selbst erweitern
glossar = datei_zu_dict("data/glossar.xlsx")
glossar = glossar | {val: key for key, val in glossar.items()}

# Einlesen von Helden
margot_data = datei_zu_dict("data/held_margot.xlsx")
margot_inv = datei_zu_dataframe("data/held_margot_inventar.xlsx")
elsbeth_data = datei_zu_dict("data/held_elsbeth.xlsx")
elsbeth_inv = datei_zu_dataframe("data/held_elsbeth_inventar.xlsx")

In [284]:
p = pd.read_excel("data/held_margot.xlsx", index_col=0)
p.dropna(inplace=True)

p.T.to_dict(orient="records")

[{'name': 'Margot',
  'rasse': 'Mensch',
  'alter': 31,
  'profession': 'Soldatin',
  'beruf': 'Stadtwache',
  'MU': 14,
  'KL': 14,
  'IN': 10,
  'CH': 10,
  'FF': 10,
  'GE': 12,
  'KO': 10,
  'KK': 12,
  'LP': 21,
  'AsP': 4,
  'KaP': 0,
  'INI': 12,
  'AP': 0,
  'geschwindigkeit': '8 schritt',
  'ausweichen': 12,
  'grimoire': 0}]

In [285]:
margot_data

{'name': 'Margot',
 'rasse': 'Mensch',
 'alter': 31,
 'profession': 'Soldatin',
 'beruf': 'Stadtwache',
 'MU': 14,
 'KL': 14,
 'IN': 10,
 'CH': 10,
 'FF': 10,
 'GE': 12,
 'KO': 10,
 'KK': 12,
 'LP': 21,
 'AsP': 4,
 'KaP': 0,
 'INI': 12,
 'AP': 0,
 'geschwindigkeit': '8 schritt',
 'ausweichen': 12,
 'grimoire': 0}

## Einheit

In [286]:
class Einheit():
    def __init__(self, heldenname: str):
        if f"data/held_{heldenname}.xlsx" is not None:
            file = datei_zu_dict(f"data/held_{heldenname}.xlsx")

            for key, value in file.items():
                setattr(self, key, value)
        
        self.inventar = datei_zu_dataframe(f"data/held_{heldenname}_inventar.xlsx")
        
        self.waffe_ausruesten()
        self.ruestung_tragen()
        self.zauber_lernen()
        self.leiteigenschaft_bestimmen()
    

    def waffe_ausruesten(self, waffe: str = None) -> None:
        """
        Der Held soll in unserem Falle nicht ohne aktive Waffe starten.
        Falls die Haupthand 'Zauber' ist, können Zauber gewirkt werden.

        : waffe : String, Name der Waffe welche ausgerüstet werden soll
        : return : None
        """

        # Beim Initialisieren des Charakters wird diese Funktion ohne zusätzlichen
        # Parameter (also waffe = None) aufgerufen. Daher wird die erste 
        if waffe == None:
            waffe = self.inventar[self.inventar["Art"] == "Waffe"].iloc[0].name

        assert waffe in dsa5_waffen.T.columns.to_list()

        framedict = zeile_zu_dict(dsa5_waffen, waffe, "Waffe")
        framedict["Kampftechnik"] = dsa5_kampftechnik.T[framedict["Art"]]

        self.haupthand = framedict


    def ruestung_tragen(self, ruestung: str = None):
        """
        
        """

        if ruestung == None:
            ruestung = self.inventar[self.inventar["Art"] == "Rüstung"].iloc[0].name

        assert ruestung in dsa5_ruestungen.T.columns.to_list()

        framedict = zeile_zu_dict(dsa5_ruestungen, ruestung, "Ruestung")
        self.ruestung = framedict
    

    def zauber_lernen(self, zauber: str = None):
        """
        
        """

        if zauber == None:
            if self.grimoire:
                self.grimoire = zeilen_zu_dict(datei_zu_dataframe(f"data/held_{self.name}_grimoire.xlsx"))
            
            else:
                self.grimoire = {}
        
        else:
            assert zauber in dsa5_zauber.T.columns, "kein bekannter Zauber!"
            assert zauber not in self.grimoire.keys, "Zauber ist schon bekannt."

            self.grimoire[zauber] = dsa5_zauber.T[zauber].to_dict()



    def leiteigenschaft_bestimmen(self):
        """
        
        """
        
        leit = self.haupthand["Kampftechnik"]["Leiteigenschaft"]

        if len(leit) == 1:
            self.leiteigenschaft = getattr(self, leit[0])

        else:
            self.leiteigenschaft = max([getattr(self, le) for le in leit])


    def _schaden_nehmen(self, schaden: int):
        """
        
        """
        
        assert schaden > 0

        self.LP -= schaden
        
        if self.LP <= 0:
            print(f"{self.name} ist kampfunfähig!")
            self.LP = 0
        
        print(f"{self.name} LP: {self.LP}")

    
    def _energie_reduzieren(self, typ: str, anzahl: int):
        """
        
        """

        if typ == "AsP":
            self.AsP -= anzahl
     
    
    def _angriffscheck(self) -> tuple:
        """
        
        """

        versuch = w20()
        zielprobe = self.leiteigenschaft + self.haupthand["Kampftechnik"]["AT"]

        erfolg = versuch < zielprobe

        return (erfolg, versuch)


    def _paradecheck(self) -> tuple:
        """
        
        """

        versuch = w20()
        zielprobe = max(self.ausweichen, self.leiteigenschaft)

        erfolg = versuch <= zielprobe

        return (erfolg, versuch)
    

    def _ausweichcheck(self) -> tuple:
        """
        
        """

        versuch = w20()
        zielprobe = self.ausweichen

        erfolg = versuch <= zielprobe

        return (erfolg, versuch)
    

    def schadensverteilung(self, other) -> int:
        """
        
        """
        
        schaden = w6(self.haupthand["TP"])
        print(f"Schaden von {self.name}: {schaden}")

        ruestungsklasse = other.ruestung["Rüstungsschutz"]
        print(f"Rüstungsklasse von {other.name}: {ruestungsklasse}")

        gemachter_schaden = [schaden - ruestungsklasse if schaden > ruestungsklasse else 0][0]

        return gemachter_schaden


    def attackieren(self, other) -> None:
        """
        
        """

        at_erfolg, at_wurf = self._angriffscheck()
        pa_erfolg, pa_wurf = other._paradecheck()

        if at_wurf == 1:
            schaden = self.schadensverteilung(other)

            print(f"Kritischer Erfolg der Angriffsprobe. {self.name} trifft {other.name} für {schaden} Schadenspunkte.")
        
        elif pa_wurf == 1:
            schaden = other.schadensverteilung(self)

            print(f"Kritischer Erfolg der Parade. {other.name} trifft {self.name} für {schaden} Schadenspunkte.")
        
        elif at_erfolg:
            if pa_erfolg:
                print(f"{other.name} pariert den erfolgreichen Angriff von {self.name}.")

                return
            
            else:
                schaden = self.schadensverteilung(other)
                print(f"{self.name} trifft {other.name} für {schaden} Schadenspunkte.")
        
        else:
            print(f"Angriff trifft {other.name} nicht.")

    
    def zaubern(self, zauber: str) -> None:
        """
        
        """

        kosten: int
        kostentyp: str

        assert zauber in self.grimoire.keys(), "unbekannter Zauber"

        kosten, kostentyp = self.grimoire[zauber]["Kosten"].split()
        kosten = int(kosten)

        if kosten > getattr(self, kostentyp):
            print(f"Nicht genügend Energie vorhanden!")

            return
        
        probe, qs = self.fertigkeitsprobe(zauber)

        self._energie_reduzieren(kostentyp, kosten)
    

    def magischer_angriff(self, other, zauber: str):
        """
        
        """

        kosten: int
        kostentyp: str

        assert zauber in self.grimoire.keys(), "unbekannter Zauber"

        kosten, kostentyp = self.grimoire[zauber]["Kosten"].split()
        kosten = int(kosten)

        if kosten > getattr(self, kostentyp):
            print(f"Nicht genügend Energie vorhanden!")

            return
        
        probe, qs = self.fertigkeitsprobe(zauber)
        aw_probe = other._ausweichcheck()[0]

        if probe:
            if aw_probe:
                print(f"{other.name} konnte dem Zauber ausweichen!")

            else:
                if self.grimoire[zauber]["Schaden"] != 0:
                    anzahl_wuerfe, wuerfel = self.grimoire[zauber]["Schaden"].split()
                    anzahl_wuerfe = int(anzahl_wuerfe)

                    schaden = w6(anzahl_wuerfe)

                    # qs_faktor = self.grimoire[zauber]["QS"]
                    qs_faktor = 2

                    ruestungsklasse = other.ruestung["Rüstungsschutz"]

                    verteilter_schaden = schaden * qs_faktor - ruestungsklasse

                    if verteilter_schaden > 0:
                        other._schaden_nehmen(verteilter_schaden)

                        print(f"{self.name} trifft {other.name} mit {zauber}.\n{other.name} erleidet {verteilter_schaden} Schadenspunkte.")

                else:
                    print(f"{zauber} richtet keinen Schaden an!")

        self._energie_reduzieren(kostentyp, kosten)


    def eigenschaftsprobe(self, probe: str, bias: int=0) -> None:
        """
        
        """
        
        wurf = w20()

        if wurf > getattr(self, glossar[probe]) + bias:
            print(f"Eigenschaftsprobe misslungen! Der nötige {probe}-Wert von {getattr(self, glossar[probe])} wurde durch {wurf} nicht unterworfen!")
        
        else:
            print(f"Eigenschaftsprobe geschafft! Der nötige {probe}-Wert von {getattr(self, glossar[probe])} wurde durch {wurf} unterworfen!")
    

    def fertigkeitsprobe(self, probe: str, bias: int=0) -> None:
        """
        
        """

        if probe in dsa5_talente.Eigenschaftsprobe.keys():
            frame = dsa5_talente
            typ = "talente"

        elif probe in dsa5_zauber.Eigenschaftsprobe.keys():
            frame = dsa5_zauber
            typ = "zauber"
        
        else:
            print("Ungültiger Input für Fertigkeitsprobe.")

            return

        ausgleich = random.randint(3, 7)
        print(f'Starte Fertigkeitsprobe für {probe} auf {frame["Eigenschaftsprobe"][probe]}, Ausgleich: {ausgleich}')


        for eigenschaft in frame["Eigenschaftsprobe"][probe]:
            wurf = w20()

            ziel = getattr(self, eigenschaft) + bias
            wurfdifferenz = ziel - wurf

            if wurfdifferenz < 0:
                ausgleich += wurfdifferenz
            
                if ausgleich < 0:
                    print(f'\nFertigkeitsprobe für {probe} misslungen!')

                    if typ == "zauber":
                        return (False, 0)
                    
                    else:
                        return
        
        print(f'\nFertigkeitsprobe für {probe} geglückt! Übriger Ausgleich: {ausgleich}')

        if typ == "zauber":
            return (True, ausgleich)

In [291]:
margot = Einheit("margot")
elsbeth = Einheit("elsbeth")
josran = Einheit("josran")

In [292]:
josran.magischer_angriff(elsbeth, "Ignifaxius")

Starte Fertigkeitsprobe für Ignifaxius auf ['MU', 'KL', 'CH'], Ausgleich: 3

Fertigkeitsprobe für Ignifaxius geglückt! Übriger Ausgleich: 3
Elsbeth LP: 6
Josran trifft Elsbeth mit Ignifaxius.
Elsbeth erleidet 18 Schadenspunkte.
